# <center>Data Mining Project 3 Spring semester 2019-2020</center>
## <center>Γεώργιος Μαραγκοζάκης &emsp; 1115201500089</center>

___

### Do all the necessary imports for this notebook

In [1]:
# data processing
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.corpus import stopwords as nltkStopwords
from string import punctuation, digits
import re
import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

# visualization
from wordcloud import WordCloud
from IPython.display import Image
from IPython.display import display
from itertools import cycle
import matplotlib.patches as mpatches

# classification
from sklearn.model_selection import KFold, cross_validate
from sklearn import svm, preprocessing
from sklearn.metrics import classification_report, make_scorer, accuracy_score, \
                            precision_score, recall_score, f1_score, roc_curve, auc,\
                            roc_auc_score, plot_roc_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pyplot as plt
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import label_binarize
import scipy
from collections import Counter
import gensim 
import random
from operator import add



# vectorization
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# clustering
from nltk.cluster import KMeansClusterer, cosine_distance
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

# for data exploration
import os
import numpy as np

## __Dataset Preprocessing__

- ### *Read all .csv files*

In [2]:
specificPath = './data/'
# find the column's names of each csv
for fileName in os.listdir(specificPath):
    # we need to check only .csv files
    if fileName.endswith(".csv"):
        thisCsv = pd.read_csv(os.path.join(specificPath, fileName), dtype='unicode')
        if fileName == 'train.csv':
            trainDF = thisCsv
        elif fileName == 'impermium_verification_labels.csv':
            testLabelsDf = thisCsv
        elif fileName == 'impermium_verification_set.csv':
            testSetDf = thisCsv
        else:
            print("Wrong File!")

__train.csv__

In [3]:
trainDF

,Insult,Date,Comment
0,1,20120618192155Z,"""You fuck your dad."""
1,0,20120528192215Z,"""i really don't understand your point.\xa0 It ..."
2,0,NaN,"""A\\xc2\\xa0majority of Canadians can and has ..."
3,0,NaN,"""listen if you dont wanna get married to a man..."
4,0,20120619094753Z,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd..."
...,...,...,...
3942,1,20120502172717Z,"""you are both morons and that is never happening"""
3943,0,20120528164814Z,"""Many toolbars include spell check, like Yahoo..."
3944,0,20120620142813Z,"""@LambeauOrWrigley\xa0\xa0@K.Moss\xa0\nSioux F..."
3945,0,20120528205648Z,"""How about Felix? He is sure turning into one ..."


__test.csv__

In [4]:
testLabelsDf

,id,Insult,Date,Comment,Usage
0,1,0,20120603163526Z,"""like this if you are a tribe fan""",PrivateTest
1,2,1,20120531215447Z,"""you're idiot.......................""",PrivateTest
2,3,1,20120823164228Z,"""I am a woman Babs, and the only ""war on women...",PrivateTest
3,4,1,20120826010752Z,"""WOW & YOU BENEFITTED SO MANY WINS THIS YEAR F...",PrivateTest
4,5,1,20120602223825Z,"""haha green me red you now loser whos winning ...",PrivateTest
...,...,...,...,...,...
2230,2231,0,20120528100303Z,"""FUCKIN LAME COME ON WTF STOP FUCKING OVER MY ...",PrivateTest
2231,2232,1,20120531185813Z,"""YOU SHUT YOUR IGNORANT PIE HOLE YOU LITTLE IN...",PrivateTest
2232,2233,0,20120529130822Z,"""sweetie pie is looking very much like her cou...",PrivateTest
2233,2234,1,20120531045826Z,"""ball4real where are you with your miami g-ayn...",PrivateTest


In [5]:
testSetDf

,id,Insult,Date,Comment,Usage
0,1,NaN,20120603163526Z,"""like this if you are a tribe fan""",PrivateTest
1,2,NaN,20120531215447Z,"""you're idiot.......................""",PrivateTest
2,3,NaN,20120823164228Z,"""I am a woman Babs, and the only ""war on women...",PrivateTest
3,4,NaN,20120826010752Z,"""WOW & YOU BENEFITTED SO MANY WINS THIS YEAR F...",PrivateTest
4,5,NaN,20120602223825Z,"""haha green me red you now loser whos winning ...",PrivateTest
...,...,...,...,...,...
2230,2231,NaN,20120528100303Z,"""FUCKIN LAME COME ON WTF STOP FUCKING OVER MY ...",PrivateTest
2231,2232,NaN,20120531185813Z,"""YOU SHUT YOUR IGNORANT PIE HOLE YOU LITTLE IN...",PrivateTest
2232,2233,NaN,20120529130822Z,"""sweetie pie is looking very much like her cou...",PrivateTest
2233,2234,NaN,20120531045826Z,"""ball4real where are you with your miami g-ayn...",PrivateTest


- ### *Convert all comments to lower case*

In [6]:
trainDF['Comment'] = trainDF['Comment'].str.lower()

trainDF

,Insult,Date,Comment
0,1,20120618192155Z,"""you fuck your dad."""
1,0,20120528192215Z,"""i really don't understand your point.\xa0 it ..."
2,0,NaN,"""a\\xc2\\xa0majority of canadians can and has ..."
3,0,NaN,"""listen if you dont wanna get married to a man..."
4,0,20120619094753Z,"""c\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd..."
...,...,...,...
3942,1,20120502172717Z,"""you are both morons and that is never happening"""
3943,0,20120528164814Z,"""many toolbars include spell check, like yahoo..."
3944,0,20120620142813Z,"""@lambeauorwrigley\xa0\xa0@k.moss\xa0\nsioux f..."
3945,0,20120528205648Z,"""how about felix? he is sure turning into one ..."


In [7]:
testLabelsDf['Comment'] = testLabelsDf['Comment'].str.lower()

testLabelsDf

,id,Insult,Date,Comment,Usage
0,1,0,20120603163526Z,"""like this if you are a tribe fan""",PrivateTest
1,2,1,20120531215447Z,"""you're idiot.......................""",PrivateTest
2,3,1,20120823164228Z,"""i am a woman babs, and the only ""war on women...",PrivateTest
3,4,1,20120826010752Z,"""wow & you benefitted so many wins this year f...",PrivateTest
4,5,1,20120602223825Z,"""haha green me red you now loser whos winning ...",PrivateTest
...,...,...,...,...,...
2230,2231,0,20120528100303Z,"""fuckin lame come on wtf stop fucking over my ...",PrivateTest
2231,2232,1,20120531185813Z,"""you shut your ignorant pie hole you little in...",PrivateTest
2232,2233,0,20120529130822Z,"""sweetie pie is looking very much like her cou...",PrivateTest
2233,2234,1,20120531045826Z,"""ball4real where are you with your miami g-ayn...",PrivateTest


In [8]:
testSetDf['Comment'] = testSetDf['Comment'].str.lower()

testSetDf

,id,Insult,Date,Comment,Usage
0,1,NaN,20120603163526Z,"""like this if you are a tribe fan""",PrivateTest
1,2,NaN,20120531215447Z,"""you're idiot.......................""",PrivateTest
2,3,NaN,20120823164228Z,"""i am a woman babs, and the only ""war on women...",PrivateTest
3,4,NaN,20120826010752Z,"""wow & you benefitted so many wins this year f...",PrivateTest
4,5,NaN,20120602223825Z,"""haha green me red you now loser whos winning ...",PrivateTest
...,...,...,...,...,...
2230,2231,NaN,20120528100303Z,"""fuckin lame come on wtf stop fucking over my ...",PrivateTest
2231,2232,NaN,20120531185813Z,"""you shut your ignorant pie hole you little in...",PrivateTest
2232,2233,NaN,20120529130822Z,"""sweetie pie is looking very much like her cou...",PrivateTest
2233,2234,NaN,20120531045826Z,"""ball4real where are you with your miami g-ayn...",PrivateTest


- ### *Delete all punctuation, urls  and characters like “\n”, “\u0111”*

Πάρθηκαν ιδέες και παραδείγματα από τα παρακάτω link:
- url detection from https://www.w3resource.com/python-exercises/re/python-re-exercise-42.php
- /+a-z0-9 detection from https://stackoverflow.com/questions/37813152/replace-words-starting-with-a-backslash-in-python
- rest /+ and punctuation from https://stackoverflow.com/questions/21672514/replacing-punctuation-in-a-data-frame-based-on-punctuation-list

In [9]:
trainDF['Comment'] = trainDF['Comment'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','', regex=True)

trainDF['Comment'] = trainDF['Comment'].str.replace(r'\\+\w+', '', regex=True)

trainDF['Comment'] = trainDF['Comment'].str.replace('[^\w\s]', '', regex=True)

trainDF

,Insult,Date,Comment
0,1,20120618192155Z,you fuck your dad
1,0,20120528192215Z,i really dont understand your point it seems t...
2,0,NaN,a of canadians can and has been wrong before n...
3,0,NaN,listen if you dont wanna get married to a man ...
4,0,20120619094753Z,c b xu bi t 2011 c ho kh ng d ng cu chi n...
...,...,...,...
3942,1,20120502172717Z,you are both morons and that is never happening
3943,0,20120528164814Z,many toolbars include spell check like yahoo f...
3944,0,20120620142813Z,lambeauorwrigleykmoss falls sd i told my boy h...
3945,0,20120528205648Z,how about felix he is sure turning into one he...


In [10]:
testLabelsDf['Comment'] = testLabelsDf['Comment'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','', regex=True)

testLabelsDf['Comment'] = testLabelsDf['Comment'].str.replace(r'\\+\w+', '', regex=True)

testLabelsDf['Comment'] = testLabelsDf['Comment'].str.replace('[^\w\s]', '', regex=True)

testLabelsDf

,id,Insult,Date,Comment,Usage
0,1,0,20120603163526Z,like this if you are a tribe fan,PrivateTest
1,2,1,20120531215447Z,youre idiot,PrivateTest
2,3,1,20120823164228Z,i am a woman babs and the only war on women i ...,PrivateTest
3,4,1,20120826010752Z,wow you benefitted so many wins this year fro...,PrivateTest
4,5,1,20120602223825Z,haha green me red you now loser whos winning n...,PrivateTest
...,...,...,...,...,...
2230,2231,0,20120528100303Z,fuckin lame come on wtf stop fucking over my b...,PrivateTest
2231,2232,1,20120531185813Z,you shut your ignorant pie hole you little ins...,PrivateTest
2232,2233,0,20120529130822Z,sweetie pie is looking very much like her cous...,PrivateTest
2233,2234,1,20120531045826Z,ball4real where are you with your miami gayness,PrivateTest


In [11]:
testSetDf['Comment'] = testSetDf['Comment'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','', regex=True)

testSetDf['Comment'] = testSetDf['Comment'].str.replace(r'\\+\w+', '', regex=True)

testSetDf['Comment'] = testSetDf['Comment'].str.replace('[^\w\s]', '', regex=True)

testSetDf

,id,Insult,Date,Comment,Usage
0,1,NaN,20120603163526Z,like this if you are a tribe fan,PrivateTest
1,2,NaN,20120531215447Z,youre idiot,PrivateTest
2,3,NaN,20120823164228Z,i am a woman babs and the only war on women i ...,PrivateTest
3,4,NaN,20120826010752Z,wow you benefitted so many wins this year fro...,PrivateTest
4,5,NaN,20120602223825Z,haha green me red you now loser whos winning n...,PrivateTest
...,...,...,...,...,...
2230,2231,NaN,20120528100303Z,fuckin lame come on wtf stop fucking over my b...,PrivateTest
2231,2232,NaN,20120531185813Z,you shut your ignorant pie hole you little ins...,PrivateTest
2232,2233,NaN,20120529130822Z,sweetie pie is looking very much like her cous...,PrivateTest
2233,2234,NaN,20120531045826Z,ball4real where are you with your miami gayness,PrivateTest


## __Classification__

- #### Classification using Naive Bayes classifier

In [12]:
def NaiveBayesClassification(trainX, trainY, testX, testY, labelEncoder):
    """
    Classify the text using the Naive Bayes classifier of scikit-learn    
    """

    clf = GaussianNB()
    
    trainX = trainX.toarray()
    
    # fit train set
    clf.fit(trainX, trainY)
    
    # Predict test set
    testX = testX.toarray()
    predY = clf.predict(testX)

    return accuracy_score(testY, predY), f1_score(testY, predY, average='weighted')

Δεύτερη συνάρτηση υλοποίησης Naive Bayes με χρήση MultinomialNB για το Laplace Smoothing.

In [13]:
def NaiveBayesClassificationLS(trainX, trainY, testX, testY, labelEncoder):
    """
    Classify the text using the Naive Bayes classifier of scikit-learn for Laplace Smoothing  
    """

    clf = MultinomialNB(alpha=1.0)
    
    trainX = trainX.toarray()
    
    # fit train set
    clf.fit(trainX, trainY)
    
    # Predict test set
    testX = testX.toarray()
    predY = clf.predict(testX)

    return accuracy_score(testY, predY), f1_score(testY, predY, average='weighted')

- #### Classification using SVM classifier

In [14]:
def SvmClassification(trainX, trainY, testX, testY, labelEncoder):
    """
    Classify the text using the SVM classifier of scikit-learn    
    """
    
    clf = svm.SVC(kernel='linear', C=1, probability=True)

    # fit train set
    clf.fit(trainX, trainY)
    
    # Predict test set
    predY = clf.predict(testX)

    return accuracy_score(testY, predY), f1_score(testY, predY, average='weighted')

- #### Classification using Random Forests classifier

In [15]:
def RandomForestClassification(trainX, trainY, testX, testY, labelEncoder):
    """
    Classify the text using the Random Forest classifier of scikit-learn    
    """
    
    clf = RandomForestClassifier()

    # fit train set
    clf.fit(trainX, trainY)
    
    # Predict test set
    predY = clf.predict(testX)

    return accuracy_score(testY, predY), f1_score(testY, predY, average='weighted')

## __Vectorization__

In [16]:
# build label encoder for Insults
le = preprocessing.LabelEncoder()
le.fit(trainDF["Insult"])

# transform Insults into numbers
trainY = le.transform(trainDF["Insult"])
testY = le.transform(testLabelsDf["Insult"])

accuracyF1Dict = dict()

Εφόσον τα Insults στο dataset μας είναι 0 ή 1 δεν χρειάζεται ο μετασχηματισμός τους μέσω του LabelEconder.

Για λόγους portability του κώδικα κρατήθηκε ο μετασχηματισμός από τη 2η εργασία.

- #### Bag-of-words vectorization

In [17]:
bowVectorizer = CountVectorizer()

trainX = bowVectorizer.fit_transform(trainDF['Comment'])
testX = bowVectorizer.transform(testSetDf['Comment'])

accuracyF1Dict["BOW-NB"] = NaiveBayesClassification(trainX, trainY, testX, testY, le)

- #### Adding Lemmatization

Η ιδέα πάρθηκε από https://scikit-learn.org/stable/modules/feature_extraction.html#customizing-the-vectorizer-classes

In [18]:
class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

In [19]:
bowVectorizer = CountVectorizer(tokenizer=LemmaTokenizer())

trainX = bowVectorizer.fit_transform(trainDF['Comment'])
testX = bowVectorizer.transform(testSetDf['Comment'])

accuracyF1Dict["BOW-NB-LM"] = NaiveBayesClassification(trainX, trainY, testX, testY, le)

- #### Removing Stop Words

In [20]:
stopWords = ENGLISH_STOP_WORDS
stopWords = (stopWords.union(nltkStopwords.words('english')))

bowVectorizer = CountVectorizer(stop_words = stopWords)

trainX = bowVectorizer.fit_transform(trainDF['Comment'])
testX = bowVectorizer.transform(testSetDf['Comment'])

accuracyF1Dict["BOW-NB-SW"] = NaiveBayesClassification(trainX, trainY, testX, testY, le)

- #### Adding Bigrams

In [21]:
bowVectorizer = CountVectorizer(ngram_range = (2, 2))

trainX = bowVectorizer.fit_transform(trainDF['Comment'])
testX = bowVectorizer.transform(testSetDf['Comment'])

accuracyF1Dict["BOW-NB-BG"] = NaiveBayesClassification(trainX, trainY, testX, testY, le)

- #### Adding Laplace Smoothing

In [22]:
bowVectorizer = CountVectorizer()

trainX = bowVectorizer.fit_transform(trainDF['Comment'])
testX = bowVectorizer.transform(testSetDf['Comment'])

accuracyF1Dict["BOW-NB-LS"] = NaiveBayesClassificationLS(trainX, trainY, testX, testY, le)

- #### Tf-idf vectorization

In [23]:
tfIdfVectorizer = TfidfVectorizer()

trainX = tfIdfVectorizer.fit_transform(trainDF['Comment'])
testX = tfIdfVectorizer.transform(testSetDf['Comment'])

accuracyF1Dict["TfIdf-SVM"] = SvmClassification(trainX, trainY, testX, testY, le)

accuracyF1Dict["TfIdf-RandomForests"] = RandomForestClassification(trainX, trainY, testX, testY, le)


- #### Adding Part-of-Speech Based Features

Συνάρτηση υπολογισμού πλήθους λέξεων με tag του τύπου:

- Noun tag = NN.*
- Verb tag = VB.*
- Adverb tag = RB.*
- Adjective tag = JJ.*

In [24]:
def countTextTag(TextTagList, tag):
    counter = 0
    for (x,y) in TextTagList:
        if y.startswith(tag):
            counter += 1
    return counter

Συνάρτηση δημιουργίας λίστας λιστών των fractions των τεσσάρων κατηγοριών λέξεων για κάθε comment.

Για τον tagger η ιδέα πάρθηκε από https://www.nltk.org/book/ch05.html 

Για τη διαίρεση με το μηδέν η ιδέα πάρθηκε από https://stackoverflow.com/questions/27317517/make-division-by-zero-equal-to-zero

In [25]:
def createFractionCharacteristics(listOfComments):
    # print("start create")
    partOfSpeechTagerList = [nltk.pos_tag(word_tokenize(comment)) for comment in listOfComments]
    nn = []
    vb = []
    rb = []
    jj = []
    for item in partOfSpeechTagerList:
        nn.append(countTextTag(item, 'NN') / len(item) if len(item) else 0)
        vb.append(countTextTag(item, 'VB') / len(item) if len(item) else 0)
        rb.append(countTextTag(item, 'RB') / len(item) if len(item) else 0)
        jj.append(countTextTag(item, 'JJ') / len(item) if len(item) else 0)
    return [nn, vb, rb, jj]

Συνάρτηση συνένωσης πίνακα tfIdf με τα 4 fractions που παρήχθησαν από το POS.

Για τη συνένωση των δύο πινάκων η ιδέα πάρθηκε από https://numpy.org/doc/stable/reference/generated/numpy.hstack.html

In [26]:
def mergeTfIdfWithPOS(textX, fractionList):
    textX = textX.toarray()
    rows, columns = textX.shape
    textX2 = np.empty([rows, columns + 4])
    for i in range(len(textX)):
        textX2[i] = np.hstack((textX[i], np.array((fractionList[0][i], fractionList[1][i], fractionList[2][i], fractionList[3][i]))))
    return textX2

In [27]:
trainX2 = mergeTfIdfWithPOS(trainX, createFractionCharacteristics(trainDF['Comment'].tolist()))

testX2 = mergeTfIdfWithPOS(testX, createFractionCharacteristics(testSetDf['Comment'].tolist()))

accuracyF1Dict["TfIdf-POS-SVM"] = SvmClassification(trainX2, trainY, testX2, testY, le)

accuracyF1Dict["TfIdf-POS-RandomForests"] = RandomForestClassification(trainX2, trainY, testX2, testY, le)

## __Beat the Benchmark__

Lets add Lemmatization to TfIdf and run it with SVM 

Από όλους τους συνδυασμούς που έκανα αυτός ήταν ο καλύτερος που οριακά έφτανε το καλύτερο μου score.

In [28]:
bowVectorizer = TfidfVectorizer(tokenizer=LemmaTokenizer())

trainX = bowVectorizer.fit_transform(trainDF['Comment'])
testX = bowVectorizer.transform(testSetDf['Comment'])

accuracyF1Dict["TfIdf-SVM-LM"] = SvmClassification(trainX, trainY, testX, testY, le)

#### Results Summary

In [29]:
resultsData = {r'Naive Bayes': ['Baseline', 'Lemmatization', 'Stop Words', 'Bigrams', 'Laplace Smoothing'],  
               'Accuracy': [accuracyF1Dict["BOW-NB"][0], accuracyF1Dict["BOW-NB-LM"][0], accuracyF1Dict["BOW-NB-SW"][0], accuracyF1Dict["BOW-NB-BG"][0], accuracyF1Dict["BOW-NB-LS"][0]],
               'F1 score': [accuracyF1Dict["BOW-NB"][1], accuracyF1Dict["BOW-NB-LM"][1], accuracyF1Dict["BOW-NB-SW"][1], accuracyF1Dict["BOW-NB-BG"][1], accuracyF1Dict["BOW-NB-LS"][1]]}

resultsData2 = {r'SVM': ['Baseline', 'TfIdf + POS'],
                'Accuracy': [accuracyF1Dict["TfIdf-SVM"][0], accuracyF1Dict["TfIdf-POS-SVM"][0]],
                'F1 Score': [accuracyF1Dict["TfIdf-SVM"][1], accuracyF1Dict["TfIdf-POS-SVM"][1]]}

resultsData3 = {r'RandomForests': ['Baseline', 'TfIdf + POS'],
                'Accuracy': [accuracyF1Dict["TfIdf-RandomForests"][0], accuracyF1Dict["TfIdf-POS-RandomForests"][0]],
                'F1 Score': [accuracyF1Dict["TfIdf-RandomForests"][1], accuracyF1Dict["TfIdf-POS-RandomForests"][1]]}

resultsData4 = {r'Beat the Benchmark': ['TfIdf + SVM + LM'],
                'Accuracy': [accuracyF1Dict["TfIdf-SVM-LM"][0]],
                'F1 Score': [accuracyF1Dict["TfIdf-SVM-LM"][1]]}

In [30]:
resultsDataFrame = pd.DataFrame(data=resultsData)
resultsDataFrame

,Naive Bayes,Accuracy,F1 score
0,Baseline,0.524832,0.524774
1,Lemmatization,0.521700,0.521326
2,Stop Words,0.525727,0.525500
3,Bigrams,0.560179,0.559484
4,Laplace Smoothing,0.688143,0.680103


In [31]:
resultsDataFrame = pd.DataFrame(data=resultsData2)
resultsDataFrame

,SVM,Accuracy,F1 Score
0,Baseline,0.697987,0.686193
1,TfIdf + POS,0.698434,0.686255


In [32]:
resultsDataFrame = pd.DataFrame(data=resultsData3)
resultsDataFrame

,RandomForests,Accuracy,F1 Score
0,Baseline,0.628635,0.579441
1,TfIdf + POS,0.623266,0.568081


In [33]:
resultsDataFrame = pd.DataFrame(data=resultsData4)
resultsDataFrame

,Beat the Benchmark,Accuracy,F1 Score
0,TfIdf + SVM + LM,0.697539,0.686238


**Συμπεράσματα**
- To dataset περιέχει πολλές λέξεις οι οποίες εκφράζουν επιθετικότητα αλλά ορισμένα γράμματα τους έχουν αντικατασταθεί από ειδικούς χαρακτήρες όπως @ και άλλοι. Επιπλέον με τον καθαρισμό του αρχείου από τους ειδικούς χαρακτήρες αυτές οι λέξεις δεν μπορούν να αναγνωριστούν πλέον.
- Η υλοποίηση των αλγορίθμων ταξινόμισης βασίζονται στην 2η εργασία.
- Η εκτέλεση των αλγορίμων SVM και RandomForest με TfIdf + Part-of-Speech χρειάζονται γύρω στα 15-17 λεπτά στο μηχάνημα μου, και αυτό προκύπτει από το μεγάλο μέγεθος των παραγόμενων πινάκων.
- Στο Part-of-Speech για κάθε μια από τις 4 κατηγορίες χαρακτηριστικών λαμβάνονται υπόψιν και παράγωγα όπως για verb είναι τα VB, VBZ και άλλα.